In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

%matplotlib inline 

# 多项式回归

In [ ]:
# 生成60对，具备一定随机性的数据集 Y = sin(x) + rand
x = np.array([i*np.pi/180 for i in range(60,300,4)])
np.random.seed(10)  #Setting seed for reproducability
y = np.sin(x) + np.random.normal(0,0.15,len(x))
data = pd.DataFrame(np.column_stack([x,y]),columns=['x','y'])
data.head(10)
plt.scatter(data['x'],data['y'])

In [ ]:
x

In [ ]:
x[1]**2

In [ ]:
# 生成多项式特征
for i in range(2,16):  
    colname = 'x_%d'%i    
    # x_2是x的平方；x_3是x的立方，以此类推
    data[colname] = data['x'] ** i

data.head()

使用RSS(残差平方和（Residual Sum of Squares))作为损失函数

In [ ]:
from sklearn.linear_model import LinearRegression

def linear_regression(data, power, models_to_plot):
    #initialize predictors:
    predictors=['x']
    if power>=2:
        predictors.extend(['x_%d'%i for i in range(2,power+1)])
        
    print(f"power = {power}--> {predictors}")
    
    #Fit the model
    linreg = LinearRegression()#normalize=True)
    linreg.fit(data[predictors],data['y'])
    # 使用fit后的模型预测得到预测值y_pred
    y_pred = linreg.predict(data[predictors])
    
    # 计算预测值y_pred和实际值y之间的RSS 残差平方和（Residual Sum of Squares）
    rss = sum((y_pred-data['y'])**2)
    # 创建了一个列表 ret，并将 RSS（残差平方和）作为第一个元素添加到列表中。
    ret = [rss]
    # 将线性回归模型的截距（intercept）添加到 ret 列表中
    ret.extend([linreg.intercept_])
    # 将线性回归模型的系数（coefficients）添加到 ret 列表中， linreg.coef_ 包含了每个特征的系数值。
    ret.extend(linreg.coef_)
    
    #Check if a plot is to be made for the entered power
    if power in models_to_plot:
#         print(f"power = {power}")
        plt.subplot(models_to_plot[power])
        plt.tight_layout()
        # 使用线plot y_pred 
        plt.plot(data['x'],y_pred,lw=3)
        # 使用点plot y 
        plt.plot(data['x'],data['y'],'.')
        
        plt.title('Plot for power: %d , RSS: %.2f'% (power,rss))
    else:
        print(f"power = {power} donot plot")
        pass
    
    return ret

In [ ]:
plt.rcParams['figure.figsize'] = 18,9

#Initialize a dataframe to store the results:
col = ['rss','intercept'] + ['coef_x_%d'%i for i in range(1,16)]
ind = ['model_pow_%d'%i for i in range(1,16)]
coef_matrix_simple = pd.DataFrame(index=ind, columns=col)

#Define the power value and plot position for which a plot is required in a dict
models_to_plot = {1:231,3:232,6:233,9:234,12:235,15:236}

#Iterate through all powers and assimilate results
for i in range(1,16):
    coef_matrix_simple.iloc[i-1,0:i+2] = linear_regression(data, power=i, models_to_plot=models_to_plot)

In [ ]:
#Set the display format to be scientific for ease of analysis
pd.options.display.float_format = '{:,.2g}'.format
coef_matrix_simple

## L2 Normalization Ridge Regression
- 岭回归是在线性回归的基础上引入L2正则化（Ridge Regression也叫L2正则化）的一种改进方法。
- 在目标函数中加入了一个惩罚项，该项是模型权重的平方和乘以一个正则化参数（alpha）。
- 这个正则化项有助于防止过拟合，尤其在特征之间存在共线性（高度相关）的情况下。


In [ ]:
from sklearn.linear_model import Ridge

def ridge_regression(data, predictors, alpha, models_to_plot={}):
    
    ridgereg = Ridge(alpha=alpha) #,normalize=True)
    ridgereg.fit(data[predictors],data['y'])
    y_pred = ridgereg.predict(data[predictors])
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([ridgereg.intercept_])
    ret.extend(ridgereg.coef_)


    #Check if a plot is to be made for the entered alpha
    if alpha in models_to_plot:
        plt.subplot(models_to_plot[alpha])
        plt.tight_layout()
        plt.plot(data['x'],y_pred,lw=3)
        plt.plot(data['x'],data['y'],'.')
        plt.title('Plot for alpha: %.3g ,RSS : %.2f'%(alpha,rss))
    return ret
        

In [ ]:
#Initialize predictors to be set of 15 powers of x
predictors=['x']
predictors.extend(['x_%d'%i for i in range(2,16)])

#Set the different values of alpha to be tested
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

#Initialize the dataframe for storing coefficients.
col = ['rss','intercept'] + ['coef_x_%d'%i for i in range(1,16)]
ind = ['alpha_%.2g'%alpha_ridge[i] for i in range(0,10)]
coef_matrix_ridge = pd.DataFrame(index=ind, columns=col)

models_to_plot = {1e-15:231, 1e-10:232, 1e-4:233, 1e-3:234, 1e-2:235, 5:236}
for i in range(10):
    coef_matrix_ridge.iloc[i,] = ridge_regression(data, predictors, alpha_ridge[i], models_to_plot)

In [ ]:
predictors

In [ ]:
pd.options.display.float_format = '{:,.2g}'.format
coef_matrix_ridge

### 有多少个系数为0

In [ ]:
coef_matrix_ridge.apply(lambda x: sum(x.values==0),axis=1)

## L1 Regulariztion Lasso Regression
- Lasso回归（Least Absolute Shrinkage and Selection Operator Regression）也是在线性回归的基础上引入正则化的方法，但是使用的是L1正则化。
- L1正则化的惩罚项是模型权重的绝对值之和乘以一个正则化参数（alpha）。
- Lasso回归有助于产生稀疏权重，即使对于大量特征，只有一部分特征的权重会非零，这可以看作是一种特征选择的方法。
- L1正则化项是模型参数的绝对值之和乘以一个正则化参数。其数学形式为：
- 这个正则化项促使模型中的某些参数趋向于零，因此具有稀疏性，可以用于特征选择。

In [ ]:
from sklearn.linear_model import Lasso

def lasso_regression(data, predictors, alpha, models_to_plot={}):
    #Fit the model
#     lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
    lassoreg = Lasso(alpha=alpha, max_iter=100000)
    lassoreg.fit(data[predictors],data['y'])
    y_pred = lassoreg.predict(data[predictors])
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    
    #Check if a plot is to be made for the entered alpha
    if alpha in models_to_plot:
        plt.subplot(models_to_plot[alpha])
        plt.tight_layout()
        plt.plot(data['x'],y_pred,lw=3)
        plt.plot(data['x'],data['y'],'.')
        plt.title('Plot for alpha: %.3g'%alpha)
    
    
    return ret

In [ ]:
#Initialize predictors to all 15 powers of x
predictors=['x']
predictors.extend(['x_%d'%i for i in range(2,16)])

#Define the alpha values to test
alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

#Initialize the dataframe to store coefficients
col = ['rss','intercept'] + ['coef_x_%d'%i for i in range(1,16)]
ind = ['alpha_%.2g'%alpha_lasso[i] for i in range(0,10)]
coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

#Define the models to plot
models_to_plot = {1e-10:231, 1e-5:232,1e-4:233, 1e-3:234, 1e-2:235, 1:236}

#Iterate over the 10 alpha values:
for i in range(10):
    coef_matrix_lasso.iloc[i,] = lasso_regression(data, predictors, alpha_lasso[i], models_to_plot)

In [ ]:
pd.options.display.float_format = '{:,.2g}'.format
coef_matrix_lasso

In [ ]:
coef_matrix_lasso.apply(lambda x: sum(x.values==0),axis=1)

正则化项是在机器学习中用于控制模型复杂度的一种技术。

它在模型的损失函数中引入一个额外的项，这个项的目的是限制模型参数的大小，从而防止过拟合。

在线性回归中，损失函数通常是最小化实际观测值与模型预测值之间的差异。正则化项添加到损失函数中，可以采用L1正则化（Lasso）或L2正则化（Ridge）。

L1正则化（Lasso）：

L1正则化项是模型参数的绝对值之和乘以一个正则化参数。其数学形式为：$$\lambda \sum_{i=1}^{n} |w_i|$$
这个正则化项促使模型中的某些参数趋向于零，因此具有稀疏性，可以用于特征选择。
L2正则化（Ridge）：

L2正则化项是模型参数的平方和乘以一个正则化参数。其数学形式为：$$\lambda \sum_{i=1}^{n} w_i^2 $$
这个正则化项对参数进行平滑，防止参数过大，有助于处理共线性问题。
正则化项的引入可以控制模型的复杂度，防止模型在训练数据上过拟合。正则化参数 \lambdaλ 是一个调整项，用于平衡模型的拟合和正则化项的影响，其值越大，正则化的影响越强。选择合适的正则化参数对于获得泛化性能好的模型非常重要。




